In [91]:
# imports
import torch
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd

import torch.optim as optim
import time
from opacus import PrivacyEngine
from vantage6.tools.util import info, warn
from torchvision import transforms
import argparse
from torchvision import datasets, transforms

In [92]:
# simple model 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


In [ ]:
# to check the params (tensors)

# model = Net()

# for parameter in model.parameters():
#     print(parameter)

In [93]:
# initialises training

def RPC_initialize_training(data, gamma, learning_rate, local_dp):
    """
    Initializes the model, optimizer and scheduler and shares the parameters
    with all the workers in the group.

    This should be sent from server to all nodes.

    Args:
        data: contains the local data from the node
        gamma: Learning rate step gamma (default: 0.7)
        learning_rate: The learning rate for training.
        cuda: Should we use CUDA?
        local_dp: bool whether to apply local_dp or not.

    Returns:
        Returns the device, model, optimizer and scheduler.
    """
    
    # Determine the device to train on
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    # print("\033[0;{};49m Rank {} is training on {}".format(device))

    # Initialize model and send parameters of server to all workers
    model = Net().to(device)

    # intializing optimizer and scheduler
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    # adding DP if true
    if local_dp == True:
        privacy_engine = PrivacyEngine(model, batch_size=64,
                sample_size=60000, alphas=range(2,32), noise_multiplier=1.3,
                max_grad_norm=1.0,)
        privacy_engine.attach(optimizer)

    # returns device, model, optimizer which will be needed in train and test
    return device, model, optimizer    

In [94]:
# basic training of the model

# Question: train gets model, device, optimizer from initialize_training, which is specified within train function, 
# why do I need to call it again before executing the function? Because in vantage6 when I sent the tasks I cannot define that but only in the master function


def RPC_train(data, log_interval, local_dp, epoch, round, delta=1e-5):
    """
    Training the model on all batches.
    Args:
        epoch: The number of the epoch the training is in.
        round: The number of the round the training is in.
        log_interval: The amount of rounds before logging intermediate loss.
        local_dp: Training with local DP?
        delta: The delta value of DP to aim for (default: 1e-5).
    """
    # loading arguments/parameters from first RPC_method
    device, model, optimizer = RPC_initialize_training(data, gamma, learning_rate, local_dp) # is this allowed in vantage6? calling one RPC_method in another?
     
    train_loader = data
    
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader): 
        # Send the data and target to the device (cpu/gpu) the model is at
        data, target = data.to(device), target.to(device)
        # Clear gradient buffers
        optimizer.zero_grad()
        # Run the model on the data
        output = model(data)
        # Calculate the loss
        loss = F.nll_loss(output, target)
        # Calculate the gradients
        loss.backward()
        # Update model
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                 100. * batch_idx / len(train_loader), loss.item()))
    
    # Adding differential privacy or not
    if local_dp == True:
        epsilon, alpha = optimizer.privacy_engine.get_privacy_spent(delta)
#             print("\033[0;{};49m Epsilon {}, best alpha {}".format(epsilon, alpha))


   

In [95]:
# Model Evaluation

def RPC_test(data):
    """
    Tests the model.

    Args:
        color: The color for the terminal output for this worker.
        model: The model to test.
        device: The device to test the model on.
        test_loader: The local loader for test local. -> no inside function
    """

    test_loader = data
    
    device, model, optimizer = RPC_initialize_training(data, gamma, learning_rate, local_dp)
     

    model.eval()
    
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # Send the local and target to the device (cpu/gpu) the model is at
            data, target = data.to(device), target.to(device)
            # Run the model on the local
            output = model(data)
            # Calculate the loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            # Check whether prediction was correct
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

    
   

In [96]:
"""
These are the parameters needed for the function
Data loading and transforming (this will be done beforehand 
and then stored in './local/training.pt' and './testing.pt')
"""

learning_rate=0.01

log_interval=10 

gamma=0.7

"""
for test set train=False!!!
"""

# train_data = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data', 
#                                                           download=True,
#                                                           train=True,
#                                                           transform=transforms.Compose([
#                                                               transforms.ToTensor(), # first, convert image to PyTorch tensor
#                                                               transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
#                                                           ])), 
#                                            batch_size=64, 
#                                            shuffle=True)

# test_data = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data', 
#                                                           download=True,
#                                                           train=False,
#                                                           transform=transforms.Compose([
#                                                               transforms.ToTensor(), # first, convert image to PyTorch tensor
#                                                               transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
#                                                           ])), 
#                                            batch_size=1000, 
#                                            shuffle=True)

train_data = torch.load("C:\\Users\\simon\\PycharmProjects\\torch-vantage6\\v6-ppsdg-py\\local\\MNIST\\processed\\training.pt")


test_data = torch.load("C:\\Users\\simon\\PycharmProjects\\torch-vantage6\\v6-ppsdg-py\\local\\MNIST\\processed\\testing.pt")


local_dp = False

epoch = 14

round = 1

In [ ]:
for epoch in range(1, epoch + 1):
    RPC_train(train_data, log_interval, local_dp, epoch, round)

RPC_test(test_data)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307038
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.276728
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.248351
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.195252
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.156293
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.109446
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.981498
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.880989
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.916348
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.725217
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.592904
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.593688
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.405067
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.393248
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.423159
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.302673
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.079327
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.225757
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.036453
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.071880
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.565970
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.427469
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.476728
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.556816
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.489850
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.530479
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.560679
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.346242
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.392650
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.432693
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.420298
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.376436
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.588904
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.411472
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.468627
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.477847
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.491620
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.490863
Train Epoch: 2 [58880/60000 (98%)]	Loss: 0.462549
Train Epoch: 2 [59520/60000 (99%)]	Loss: 0.403723


Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.573603
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.413333
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.518023
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.565629
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.618966
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.308446
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.483383
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.490562
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.634947
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.500235
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.480866
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.387021
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.536208
Train Epoch: 4 [42880/60000 (71%)]	Loss: 0.362595
Train Epoch: 4 [43520/60000 (72%)]	Loss: 0.352278
Train Epoch: 4 [44160/60000 (74%)]	Loss: 0.399202
Train Epoch: 4 [44800/60000 (75%)]	Loss: 0.507775
Train Epoch: 4 [45440/60000 (76%)]	Loss: 0.537993
Train Epoch: 4 [46080/60000 (77%)]	Loss: 0.418124
Train Epoch: 4 [46720/60000 (78%)]	Loss: 0.616988


Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.456248
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.560669
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.732290
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.674229
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.508435
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.598273
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.621956
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.782022
Train Epoch: 6 [26880/60000 (45%)]	Loss: 0.694544
Train Epoch: 6 [27520/60000 (46%)]	Loss: 0.645464
Train Epoch: 6 [28160/60000 (47%)]	Loss: 0.382226
Train Epoch: 6 [28800/60000 (48%)]	Loss: 0.510860
Train Epoch: 6 [29440/60000 (49%)]	Loss: 0.692370
Train Epoch: 6 [30080/60000 (50%)]	Loss: 0.453091
Train Epoch: 6 [30720/60000 (51%)]	Loss: 0.590729
Train Epoch: 6 [31360/60000 (52%)]	Loss: 0.405791
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.734785
Train Epoch: 6 [32640/60000 (54%)]	Loss: 0.377478
Train Epoch: 6 [33280/60000 (55%)]	Loss: 0.363326
Train Epoch: 6 [33920/60000 (57%)]	Loss: 0.396711


Train Epoch: 8 [7680/60000 (13%)]	Loss: 1.166023
Train Epoch: 8 [8320/60000 (14%)]	Loss: 1.182602
Train Epoch: 8 [8960/60000 (15%)]	Loss: 1.052507
Train Epoch: 8 [9600/60000 (16%)]	Loss: 1.119678
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.924623
Train Epoch: 8 [10880/60000 (18%)]	Loss: 0.726707
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.918832
Train Epoch: 8 [12160/60000 (20%)]	Loss: 0.845339
Train Epoch: 8 [12800/60000 (21%)]	Loss: 0.868111
Train Epoch: 8 [13440/60000 (22%)]	Loss: 0.731411
Train Epoch: 8 [14080/60000 (23%)]	Loss: 0.761909
Train Epoch: 8 [14720/60000 (25%)]	Loss: 0.661866
Train Epoch: 8 [15360/60000 (26%)]	Loss: 0.586607
Train Epoch: 8 [16000/60000 (27%)]	Loss: 0.779190
Train Epoch: 8 [16640/60000 (28%)]	Loss: 0.677737
Train Epoch: 8 [17280/60000 (29%)]	Loss: 0.704568
Train Epoch: 8 [17920/60000 (30%)]	Loss: 0.791850
Train Epoch: 8 [18560/60000 (31%)]	Loss: 0.622774
Train Epoch: 8 [19200/60000 (32%)]	Loss: 0.828409
Train Epoch: 8 [19840/60000 (33%)]	Loss: 0.551936
Trai

Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.494191
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.388006
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.326574
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.410755
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.141890
Train Epoch: 9 [56320/60000 (94%)]	Loss: 0.350835
Train Epoch: 9 [56960/60000 (95%)]	Loss: 0.316209
Train Epoch: 9 [57600/60000 (96%)]	Loss: 0.432180
Train Epoch: 9 [58240/60000 (97%)]	Loss: 0.442706
Train Epoch: 9 [58880/60000 (98%)]	Loss: 0.428442
Train Epoch: 9 [59520/60000 (99%)]	Loss: 0.347755
Train Epoch: 10 [0/60000 (0%)]	Loss: 2.300072
Train Epoch: 10 [640/60000 (1%)]	Loss: 2.273347
Train Epoch: 10 [1280/60000 (2%)]	Loss: 2.257280
Train Epoch: 10 [1920/60000 (3%)]	Loss: 2.197731
Train Epoch: 10 [2560/60000 (4%)]	Loss: 2.174198
Train Epoch: 10 [3200/60000 (5%)]	Loss: 2.100051
Train Epoch: 10 [3840/60000 (6%)]	Loss: 2.057113
Train Epoch: 10 [4480/60000 (7%)]	Loss: 2.029469
Train Epoch: 10 [5120/60000 (9%)]	Loss: 1.914846
Train Epoch: 

Train Epoch: 11 [36480/60000 (61%)]	Loss: 0.422243
Train Epoch: 11 [37120/60000 (62%)]	Loss: 0.477031
Train Epoch: 11 [37760/60000 (63%)]	Loss: 0.646639
Train Epoch: 11 [38400/60000 (64%)]	Loss: 0.526048
Train Epoch: 11 [39040/60000 (65%)]	Loss: 0.422989
Train Epoch: 11 [39680/60000 (66%)]	Loss: 0.528404
Train Epoch: 11 [40320/60000 (67%)]	Loss: 0.488902
Train Epoch: 11 [40960/60000 (68%)]	Loss: 0.557249
Train Epoch: 11 [41600/60000 (69%)]	Loss: 0.386858
Train Epoch: 11 [42240/60000 (70%)]	Loss: 0.376154
Train Epoch: 11 [42880/60000 (71%)]	Loss: 0.515761
Train Epoch: 11 [43520/60000 (72%)]	Loss: 0.372912
Train Epoch: 11 [44160/60000 (74%)]	Loss: 0.366065
Train Epoch: 11 [44800/60000 (75%)]	Loss: 0.458043
Train Epoch: 11 [45440/60000 (76%)]	Loss: 0.374210
Train Epoch: 11 [46080/60000 (77%)]	Loss: 0.374141
Train Epoch: 11 [46720/60000 (78%)]	Loss: 0.323299
Train Epoch: 11 [47360/60000 (79%)]	Loss: 0.267860
Train Epoch: 11 [48000/60000 (80%)]	Loss: 0.234048
Train Epoch: 11 [48640/60000 (8

# FED AVG

In [ ]:
# FedAvg gathering of parameters 

def RPC_get_parameters(data, model, parameters):
    """
    Get parameters from nodes
    """
    data_size = len(data) // 3 # number of nodes# size of dataset
    
    RPC_train(data, log_interval, local_dp, epoch, round)

    with torch.no_grad():
        for parameters in model.parameters():
            return {"params": parameters}

In [ ]:
# averaging of returned parameters 

def average_parameters(data, model):
    """
    Get parameters from nodes and calculate the average
    :param model: torch model
    :param parameters: parameters of model
    :param weights:
    :return:
    """
    
    parameters = RPC_get_parameters(data, model, parameters) # makes returned parameters from RPC_get_parameters the parameters used in this function

    # TODO: local: since we usually just get the parameters, this well be an entire task, therefore, we might need to train for each individually


    with torch.no_grad():
        for parameters in model.parameters():
            average = sum(x * y for x, y in zip(parameters[i], weights)) / sum(weights)
            parameters.data = average
            i = i + 1
        return {
            "params_averaged": model
        }
    

#     i = 0
#     with torch.no_grad():
#     for param in model.parameters():
#     # The first entry of the provided parameters when using dist.gather
#     # method also contains the value from the server, remove that one
#     minus_server = parameters[i][1:]
#     # Calculate the average by summing and dividing by the number of
#     # workers
#     s = sum(minus_server)
#     average = s/len(minus_server)
#     # Change the parameter of the global model to the average
#     param.data = average
#     i = i + 1

In [ ]:
data = torch.load("C:\\Users\\simon\\PycharmProjects\\torch-vantage6\\v6-ppsdg-py\\local\\MNIST\\processed\\training.pt")
model = Net()
parameters=model.parameters()
learning_rate=0.01
log_interval=10 
gamma=0.7
round = 1
epoch =1

RPC_get_parameters(data, model, parameters)

In [ ]:
average_parameters(data, model)

In [ ]:
# training with those averaged parameters

def RPC_fed_avg(data, local_dp, epoch, delta=1e-5):
    """
    Training and testing the model on the workers concurrently using federated
    averaging, which means calculating the average of the local model
    parameters after a number of (local) epochs each training round.

    In vantage6, this method will be the training of the model with the average parameters (weighted)

    Returns:
        Returns the final model
    """
    # TODO: local: since we usually just get the parameters, this well be an entire task, therefore, we might need to train for each individually
    model = average_parameters(data, model)
    
    for epoch in range(1, epoch + 1):
        # Train the model on the workers again
        RPC_train(data, local_dp, model, device, optimizer, epoch, delta=1e-5)
        # Test the model on the workers
        RPC_test(data, model, device)

    gather_params = model.get_parameters() # or model.parameters()

    RPC_train(model.RPC_average_parameters_weighted(gather_params))

    return model, parameters


    ## OR 

#     parameters = RPC_average_parameters_weighted(data, model, parameters, weights) # then uses those parameters for training



        # # Gather the parameters after the training round on the server
        #     gather_params = coor.gather_parameters(rank, model, group_size + 1, subgroup)
        #
        #     # If the server
        #     if rank == 0:
        #         # Calculate the average of the parameters and adjust global model
        #         coor.average_parameters_weighted(model, gather_params, weights)
        #
        #     # Send the new model parameters to the workers
        #     coor.broadcast_parameters(model, group)


In [ ]:
RPC_fed_avg(data, local_dp, epoch, delta=1e-5)